In [10]:
import boto3
import pandas as pd
from datetime import datetime

In [2]:
s3 = boto3.client('s3')

In [5]:
print(s3.list_buckets()['Buckets'])

[{'Name': 'testlzy', 'CreationDate': datetime.datetime(2021, 12, 20, 1, 44, 6, tzinfo=tzutc())}]


In [6]:
if 'tdf-tht' not in [n['Name'] for n in s3.list_buckets()['Buckets']]:
    s3.create_bucket(
                ACL='public-read',
                Bucket='tdf-tht',
                CreateBucketConfiguration={'LocationConstraint': 'ap-southeast-2'}
            )
    print('The new bucket (tdf-tht) has been created.')
else :
    print('The bucket (tdf-tht) exists, please proceeding further work.')

The new bucket (tdf-tht) has been created.


In [7]:
import requests
cityName = 'Sydney'
APIKey = 
response = requests.get(f"http://api.openweathermap.org/data/2.5/weather?q={cityName}&units=metric&appid={APIKey}").json()
print(response)

{'coord': {'lon': 151.2073, 'lat': -33.8679}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03d'}], 'base': 'stations', 'main': {'temp': 28.18, 'feels_like': 30.88, 'temp_min': 25.76, 'temp_max': 31.14, 'pressure': 1009, 'humidity': 69}, 'visibility': 10000, 'wind': {'speed': 1.34, 'deg': 164, 'gust': 2.68}, 'clouds': {'all': 42}, 'dt': 1639964787, 'sys': {'type': 2, 'id': 2018875, 'country': 'AU', 'sunrise': 1639939224, 'sunset': 1639991087}, 'timezone': 39600, 'id': 2147714, 'name': 'Sydney', 'cod': 200}


In [8]:
try:
    s3.get_object(Bucket='tdf-tht', Key='Staging/weatherDataHourly.csv')
except :
    head = 'weather,temp,temp_min,temp_max,pressure,humidity,visibility,datetime,sunrise,sunset'
    s3.put_object(Body=head, Bucket='tdf-tht', Key='Staging/weatherDataHourly.csv')

In [11]:
{'weather':response['weather'][0]['main'], 
           'temp':response['main']['temp'], 
           'temp_min':response['main']['temp_min'], 
           'temp_max':response['main']['temp_max'],
           'pressure':response['main']['pressure'], 
           'humidity':response['main']['humidity'],
           'visibility':response['visibility'],
           'datetime':datetime.fromtimestamp(response['dt']),
           'sunrise':datetime.fromtimestamp(response['sys']['sunrise']), 
           'sunset':datetime.fromtimestamp(response['sys']['sunset'])}

{'weather': 'Clouds',
 'temp': 28.18,
 'temp_min': 25.76,
 'temp_max': 31.14,
 'pressure': 1009,
 'humidity': 69,
 'visibility': 10000,
 'datetime': datetime.datetime(2021, 12, 20, 12, 46, 27),
 'sunrise': datetime.datetime(2021, 12, 20, 5, 40, 24),
 'sunset': datetime.datetime(2021, 12, 20, 20, 4, 47)}

In [15]:
s3.download_file('tdf-tht', 'Staging/weatherDataHourly.csv','file_name')
df = pd.read_csv('file_name')
df

,weather,temp,temp_min,temp_max,pressure,humidity,visibility,datetime,sunrise,sunset
0,Clouds,28.18,25.76,31.14,1009,69,10000,2021-12-20 12:46:27,2021-12-20 05:40:24,2021-12-20 20:04:47


In [14]:
s3.download_file('tdf-tht', 'Staging/weatherDataHourly.csv','file_name')
df = pd.read_csv('file_name')
new_row = {'weather':response['weather'][0]['main'], 
           'temp':response['main']['temp'], 
           'temp_min':response['main']['temp_min'], 
           'temp_max':response['main']['temp_max'],
           'pressure':response['main']['pressure'], 
           'humidity':response['main']['humidity'],
           'visibility':response['visibility'],
           'datetime':datetime.fromtimestamp(response['dt']),
           'sunrise':datetime.fromtimestamp(response['sys']['sunrise']), 
           'sunset':datetime.fromtimestamp(response['sys']['sunset'])}
df = df.append(new_row, ignore_index=True)
bytes_to_write = df.to_csv(None, index=False).encode()
s3.put_object(Body=bytes_to_write, Bucket='tdf-tht', Key='Staging/weatherDataHourly.csv')

{'ResponseMetadata': {'RequestId': 'ABNKB49AP12NSM7V',
  'HostId': 'lfucuXycee6NBTd9RSqeIfIIBy5E6XITrfOb+pyzEsrr0UbBalGol/X3qYp4GQ9Ne6g4zP4J1tE=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'lfucuXycee6NBTd9RSqeIfIIBy5E6XITrfOb+pyzEsrr0UbBalGol/X3qYp4GQ9Ne6g4zP4J1tE=',
   'x-amz-request-id': 'ABNKB49AP12NSM7V',
   'date': 'Mon, 20 Dec 2021 02:07:06 GMT',
   'etag': '"ee08d298d08f53ad68a7eeba9d8f9dd2"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"ee08d298d08f53ad68a7eeba9d8f9dd2"'}